<a href="https://colab.research.google.com/github/Ashwath26112006/Backend_Task2_MozillaFirefox/blob/main/backend_task2_to_do_list_api_mozillafirefox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import flask
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from werkzeug.security import generate_password_hash, check_password_hash
import jwt
import datetime
from functools import wraps
import socket
from gunicorn.app.base import BaseApplication
from multiprocessing import Process, Queue
import time

# Initialize Flask app and configure database
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///todo.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define User model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(50), unique=True, nullable=False)
    password = db.Column(db.String(100), nullable=False)
    tasks = db.relationship('Task', backref='user', lazy=True)

# Define Task model
class Task(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    description = db.Column(db.String(200))
    done = db.Column(db.Boolean, default=False)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

# Decorator for token-based authentication
def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = request.headers.get('Authorization')
        if not token:
            return jsonify({'message': 'Token is missing!'}), 401
        try:
            data = jwt.decode(token, app.config['SECRET_KEY'], algorithms=["HS256"])
            current_user = User.query.filter_by(id=data['user_id']).first()
        except:
            return jsonify({'message': 'Token is invalid!'}), 401
        return f(current_user, *args, **kwargs)
    return decorated

# Route for user registration
@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = generate_password_hash(data['password'], method='sha256')
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message': 'New user created!'}), 201

# Route for user login
@app.route('/login', methods=['POST'])
def login():
    auth = request.authorization
    if not auth or not auth.username or not auth.password:
        return jsonify({'message': 'Could not verify'}), 401
    user = User.query.filter_by(username=auth.username).first()
    if not user:
        return jsonify({'message': 'User not found'}), 401
    if check_password_hash(user.password, auth.password):
        token = jwt.encode({'user_id': user.id, 'exp': datetime.datetime.utcnow() + datetime.timedelta(minutes=30)}, app.config['SECRET_KEY'])
        return jsonify({'token': token})
    return jsonify({'message': 'Could not verify'}), 401

# Route to get all tasks for a user
@app.route('/tasks', methods=['GET'])
@token_required
def get_all_tasks(current_user):
    tasks = Task.query.filter_by(user_id=current_user.id).all()
    return jsonify({'tasks': [{'id': task.id, 'title': task.title, 'description': task.description, 'done': task.done} for task in tasks]})

# Route to create a new task
@app.route('/tasks', methods=['POST'])
@token_required
def create_task(current_user):
    data = request.get_json()
    new_task = Task(title=data['title'], description=data['description'], user_id=current_user.id)
    db.session.add(new_task)
    db.session.commit()
    return jsonify({'message': 'New task created!'}), 201

# Route to update or delete a task
@app.route('/tasks/<int:task_id>', methods=['PUT', 'DELETE'])
@token_required
def manage_task(current_user, task_id):
    task = Task.query.filter_by(id=task_id, user_id=current_user.id).first()
    if not task:
        return jsonify({'message': 'No task found!'}), 404
    if request.method == 'PUT':
        data = request.get_json()
        task.title = data.get('title', task.title)
        task.description = data.get('description', task.description)
        task.done = data.get('done', task.done)
        db.session.commit()
        return jsonify({'message': 'Task updated!'})
    elif request.method == 'DELETE':
        db.session.delete(task)
        db.session.commit()
        return jsonify({'message': 'Task deleted!'})

# Function to find a free port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]

# Function to run the app with Gunicorn
def run_app(queue):
    class FlaskApplication(BaseApplication):
        def __init__(self, app, options=None):
            self.application = app
            self.options = options or {}
            super().__init__()

        def load_config(self):
            for key, value in self.options.items():
                self.cfg.set(key, value)

        def load(self):
            return self.application

    port = find_free_port()
    queue.put(port)
    options = {
        'bind': f'0.0.0.0:{port}',
        'workers': 1,
    }
    print(f"Starting server on port {port}")
    FlaskApplication(app, options).run()

# Main execution
if __name__ == '__main__':
    # Create database tables
    with app.app_context():
        db.create_all()

    # Start the server in a separate process
    port_queue = Queue()
    server_process = Process(target=run_app, args=(port_queue,))
    server_process.start()

    print("Waiting for server to start...")
    time.sleep(5)

    # Get the port number from the queue
    server_port = port_queue.get()
    print(f"Server is running on port {server_port}")

    # Keep the main process running
    server_process.join()


Starting server on port 39621


[2025-03-04 14:18:11 +0000] [9316] [INFO] Starting gunicorn 23.0.0
[2025-03-04 14:18:11 +0000] [9316] [INFO] Listening at: http://0.0.0.0:39621 (9316)
[2025-03-04 14:18:11 +0000] [9316] [INFO] Using worker: sync
[2025-03-04 14:18:11 +0000] [9328] [INFO] Booting worker with pid: 9328


Waiting for server to start...
Server is running on port 39621
